In [ ]:
import base64
import io
import numpy as np
from matplotlib import pyplot as plt

from PIL import Image, ImageDraw
from skimage.draw import polygon
from skimage.measure import regionprops, label, find_contours
from skimage.morphology import medial_axis

In [ ]:
imsize = 50

In [ ]:
img64 = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAfQAAAH0CAYAAADL1t+KAAAgAElEQVR4Xu3di5UkxbEG4JAFd7FAYAFgAcgCwAIWCxYsAFnAYgHCAsAChAWABYAFCAt0T8C0mN2Z2anKrkdG5tfn7FnuVVU+vsjuf7u6Hn8LLwIECBAgQKC8wN/Kz8AECBAgQIAAgRDoFgEBAgQIEBhAQKAPUERTIECAAAECAt0aIECAAAECAwgI9AGKaAoECBAgQECgWwMECBAgQGAAAYE+QBFNgQABAgQICHRrgAABAgQIDCAg0AcooikQIECAAAGBbg0QIECAAIEBBAT6AEU0BQIECBAgINCtAQIECBAgMICAQB+giKZAgAABAgQEujVAgAABAgQGEBDoAxTRFAgQIECAgEC3BggQIECAwAACAn2AIpoCAQIECBAQ6NYAAQIECBAYQECgD1BEUyBAgAABAgLdGiBAgAABAgMICPQBimgKBAgQIEBAoFsDBAgQIEBgAAGBPkARTYEAAQIECAh0a4AAAQIECAwgINAHKKIpECBAgAABgW4NECBAgACBAQQE+gBFNAUCBAgQICDQrQECBAgQIDCAgEAfoIimQIAAAQIEBLo1QIAAAQIEBhAQ6AMU0RQIECBAgIBAtwYIECBAgMAAAgJ9gCKaAgECBAgQEOjWAAECBAgQGEBAoA9QRFMgQIAAAQIC3RogQIAAAQIDCAj0AYpoCgQIECBAQKBbAwQIECBAYAABgT5AEU2BAAECBAgIdGuAAAECBAgMICDQByiiKRAgQIAAAYFuDRAgQIAAgQEEBPoARTQFAgQIECAg0K0BAgQIECAwgIBAH6CIpkCAAAECBAS6NUCAAAECBAYQEOgDFNEUCBAgQICAQLcGCBAgQIDAAAICfYAimgIBAgQIEBDo1gABAgQIEBhAQKAPUERTIECAAAECAt0aIECAAAECAwgI9AGKaAoECBAgQECgWwMECBAgQGAAAYE+QBFNgQABAgQICHRrgAABAgQIDCAg0AcooikQIECAAAGBbg0QIECAAIEBBAT6AEU0BQIECBAgINCtAQIECBAgMICAQB+giKZAgAABAgQEujVAgAABAgQGEBDoAxTRFAgQIECAgEC3BggQIECAwAACAn2AIpoCAQIECBAQ6NYAAQIECBAYQECgD1BEUyBAgAABAgLdGiBAgAABAgMICPQBimgKBAgQIEBAoFsDBAgQIEBgAAGBPkARTYEAAQIECAh0a4AAAQIECAwgINAHKKIpECBAgAABgW4NECBAgACBAQQE+gBFNAUCBAgQICDQrQECBAgQIDCAgEAfoIimQIAAAQIEBLo1QIAAAQIEBhAQ6AMU0RQIECBAgIBAtwYIECBAgMAAAgJ9gCKaAgECBAgQEOjWwJECTyLizYjIv996qeN3b/3fP0bEf+4Z2OX//3tE5H97ESBAgMCNgEC3FPYSyNB+JyIyqDO880/+/7Z+/RIR+ecS9pe/v9+6I+0RIECgZwGB3nN16o0tQ/vDWyF+9gzyW34G/CXk/x0Rv978A+DssemfAAECmwoI9E05p2zs9ZsQfxoR+d9VXhnyl2/2l2/5Pz1wqL/KnIyTAIGJBQT6xMW/cup5KP3Tm2/jVzbV1e6Xb/WXkPetvqvyGAwBAg8JCHRrY61AHlL/+J6T2ta2U3H7DPfbQe8bfcUqGjOBQQUE+qCF3WFa70fE58UOq+/AcKfJy2H7PISfgS/kj1DXBwECdwQEukXxmECe6JZBfvuyssf2mf1/v4R8BnwGvTPuZ18R5k/gAAGBfgBy0S7yErP8jTwPr3tdL3A52/7yt5C/3lQLBAjcEhDolsN9Avmt/GuH13dfHJdv8Jcz7oX87uQ6IDCugEAft7atM3sWEc9bd7bf1QKXb/B52N4Z9ldzaoDAPAICfZ5aPzbTPMSev5Xn9eRefQncvkHO5fd5J9/1VSOjIXC6gEA/vQRdDCDD/LtJL0XrogCNg3j5mvnb98B3+L4R1W4EqgoI9KqV227cwnw7Sy31J3C5b8Dtkd3+h8/t/90/gvqrnxGtEBDoK7AG3fQH38wHraxptQpcQv5yBOTyt6f8tYra7xABgX4Ic7ed5MlveRKcFwECywVuP+HPOQ3L3Wy5s4BA3xm44+bzzm95aZoXAQLbCLx810CH8Ldx1cpCAYG+EGqwzfJ38zzUXunpaIOVwHQmEbjcayD/zoDPw/deBHYREOi7sHbfqEPt3ZfIAAcVuNzzX8APWuAzpyXQz9Q/p+/8Vv7zOV3rlQCBlwQy4L+5uYmQQ/SWx1UCAv0qvpI7/ysi8hGoXgQI9CeQ39wvfwR8f/XpekQCvevybD64/O38t81b1SABAnsJCPi9ZAdsV6APWNRXTOmzmyeozTVrsyUwjsAl4L+9eTTvODMzk6sFBPrVhKUayN/OndleqmQGS+BBgTxjPgM+f4PPw/N52ZzXxAICfZ7iu+58nlqb6ZwClzPoLwE/p8LEsxbo8xTfyXDz1NpMCeS39wz2S7i7/n2CNSHQJyhyRDgZbo46myWBhwQu4Z6/vQv3QdeJQB+0sC9NK59x/uUcUzVLAgQeERDugy4RgT5oYV+aVr6B35tjqmZJgMBCgcth+S+cMb9QrPPNBHrnBdpgeA63b4CoCQKDC+TZ8h85U752lQV67fotGb3D7UuUbEOAQArkcx4+QVFTQKDXrNuaUTvcvkbLtgQI5GdGflt38lyxtSDQixVs5XAdbl8JZnMCBP4QyEPw/2BRS0Cg16rX2tE63L5WzPYECFwEPri5jp1IEQGBXqRQjcN0uL0Rzm4ECPxxyP01DnUEBHqdWq0dqcPta8VsT4DAywK+pRdaEwK9ULFWDtW921eC2ZwAgTsCX0VE/nTnVUBAoBcoUuMQ3bu9Ec5uBAj8TyAf+PI2jxoCAr1GnVpG+dvNPdxb9rUPAQIELgJyoshaUKgihVo5TIfbV4LZnACBBwXkRJHFoVBFCrVymA63rwSzOQECDwrk9eh5XbpX5wICvfMCNQ7v54h4vXFfuxEgQOC2gEAvsh4EepFCrRjmWxHxw4rtbUqAAIFXCeRJcXlynFfnAgK98wI1DC8frvCsYT+7ECBA4D4BOVFkXShUkUKtGGZ+O89v6V4ECBDYQkBObKF4QBsKdQDygV3k7+b5+7kXAQIEthD41fk4WzAe04ZAP8b5qF4+jojPj+pMPwQIDC/wbUTkZbBeBQQEeoEirRiih7GswLIpAQKPCvwzIj57dCsbdCEg0Lsow2aD+O9mLWmIAAECER7OUmgVCPRCxXpkqO4ON04tzYRALwL5+NR8jKpXAQGBXqBIC4eYh8U+XbitzQgQIPCYwE+umHmMqK//XaD3VY9rRpO3ZnznmgbsS4AAgVsCX0REnmjrVURAoBcp1IJherraAiSbECCwWMAd4hZT9bGhQO+jDteO4klEZKB7ESBAYAsB159voXhwGwL9YPCduns3Ir7bqW3NEiAwn4DL1QrWXKAXLNo9QxboY9TRLAj0IuDs9l4qsWIcAn0FVsebOsO94+IYGoFiAl9FxNNiYzbciBDoYywDgT5GHc2CQA8Cb0TELz0MxBjWCQj0dV69bi3Qe62McRGoJeDbea16vTBagV64eLeGnofHvhxjKmZBgMBJAr/fPFnNneFOKsC13Qr0awX72N9JcX3UwSgIVBb4JCKeV57A7GMX6GOsAM9BH6OOZkHgLIHvIyK/GHgVFhDohYv30tA9aW2cWpoJgSMFHGo/UnvHvgT6jrgHN+1e7geD647AAAIZ5vnN/McB5jL9FAT6OEsgH6Lw+TjTMRMCBA4Q8LzzA5CP6kKgHyW9fz9+R9/fWA8ERhL4KCL+NdKEZp+LQB9rBTjsPlY9zYbAHgJ5mD0vdf1mj8a1eZ6AQD/Pfo+eXb62h6o2CYwj4DfzcWp5ZyYCfbzi5r+63xtvWmZEgMCVAj/dnADnxjFXQva6u0DvtTLt48pno+d9mP+vvQl7EiAwmMAXEZEnznoNLCDQxyzuWxGRv6cL9THra1YElgrkIfb3bz4Plu5ju6ICAr1o4RYMW6gvQLIJgYEF8lt5PrjJIfaBi3x7agJ97ELnpWx5Wco7Y0/T7AgQuCXw681Z7HmUzmsiAYE+R7HzX+n5+5lD8HPU2yznFMjD6/le94CVOesfAn2ewufJcvlmfzbPlM2UwDQC+Rzz/Ee7w+vTlPzuRAX6fMXPw/B5U4n88/f5pm/GBIYSyCDPf6jnlS1ekwsI9LkXQJ79msHuuvW514HZ1xMQ5PVqtvuIBfruxCU6yMPxGe55p7n822/tJcpmkJMJ5G/keZJr/kbuG/lkxV8yXYG+RGm+bfKStwz3/JP/7dD8fGvAjPsR+P4myPMukH4j76cu3Y1EoHdXki4HlN/gM9jzz+W/8+/882aXIzYoArUF8jat+W08Q9y38dq1PGz0Av0w6uE7ypPt8s/lld/uL6+X/zfXxQ+/HExwpUBeO57XjWeA59++ia8EtHm4bM0iOF3gdthfvv3noC7/f0cBTi+RAewgcAnwDO/841v4DsizNekb+mwVrzvfS9hn0F9+43e4v249Zxu5AJ+t4ifMV6CfgK7LzQQy3POs/LyhhhP3NmPV0AYC+Rt4fvP+0TfwDTQ1sUhAoC9islEBgQz1vMGGS+4KFGvAIeaZ6Jfw9hv4gAWuMCWBXqFKxrhUIL+x50lFDsUvFbNdi8Dl8HkG+CXEW9qxD4FNBQT6ppwa60Agf2vPG2982MFYDKG+QB46z9DOk9Yuh9CdgV6/rkPOQKAPWVaTurmGV6hbCksF8pB5hnb+uQR4/u1FoIyAQC9TKgNtEMgbcwj1BriBdsnD45dLwvKb9SWkL88K98zwgYo9+1QE+uwrYOz55+H3/MD2m/p4dc5v1JeAvh3Ul2/Z483YjAg8IiDQLZHRBTLU80Pe2e91K337Gu7LiWh1Z2PkBHYSEOg7wWq2K4G8Vv3rrkZkMI8JfHvrNqjuovaYlv+dQLj1q0Uwj0Ce+f5snumWnOklxD1VrGT5DPpsAd/Qz66A/o8ScOj9KOl1/eQzvvMfW3kCo2/i6+xsTeAFAYFuQcwk8DQivpxpwh3PNX8Xzzv7+TbecZEMrZaAQK9VL6O9XiDPevf41usdW1u4fCPPb+Vu0NKqaD8C9wgIdMtiNoF8Tvt3s026k/nmpWZ5lMSh9U4KYhhjCQj0seppNssEfEtf5rTVVvmtPA+v57dyLwIEdhIQ6DvBarZrAd/SjytP/laelw26jepx5nqaVECgT1p40/7jsK9nqO+7EPLBJvmPJ7+V7+usdQJ/CAh0C2FWgXx++uezTv6Aeec15fl7uTA/AFsXBAS6NTCzQF6X/tvMADvOPcM8D7N7ESBwoIBv6Adi66o7AU9j274kDrNvb6pFAosEBPoiJhsNKuAe79sWNs9mf91h9m1RtUZgqYBAXyplu1EF8jdeT2Lbprr/uHlc7TataYUAgVUCAn0Vl40HFHDYfZui5k1j8ox2LwIEThIQ6CfB67YbAYfdtynF26413wZSKwRaBQR6q5z9RhFwtvv1lcybx+Rv514ECJwoINBPxNd1NwJuBXtdKb6IiLyu34sAgRMFBPqJ+LruRsBNZq4rxUc3zzO/rhV7EyBwlYBAv4rPzoMIvBURPwwylzOm4ez2M9T1SeAlAYFuSRD4U8Dla+0r4TXXnrfj2ZPAVgICfStJ7VQX8Dt6ewV9jrTb2ZPAZgLeiJtRaqi4QD6v+9Piczhr+D5HzpLXL4FbAt6IlgOBPwVcj962EtxQps3NXgQ2FxDom5NqsKhAXkf9c9GxnzlsgX6mvr4J+IZuDRC4V+C/XFYLCPTVZHYgsI+Ab+j7uGq1poAT49bX7Z8RkecfeBEgcLKAQD+5ALrvSsCDWtaXQ6CvN7MHgV0EBPourBotKuBM9/WFE+jrzexBYBcBgb4Lq0aLCjyNiC+Ljv2sYX8QEd+c1bl+CRD4S0CgWw0E/hLI53l/B2SVgNu+ruKyMYH9BAT6frZaricg0NfXTKCvN7MHgV0EBPourBotLODStXXFeyMiflm3i60JENhDQKDvoarNygICfV31fIas87I1gd0EvBl3o9VwUQFPXVtXOJ8h67xsTWA3AW/G3Wg1XFTAzWWWF+73iHiyfHNbEiCwp4BA31NX2xUFBPryqrnt63IrWxLYXUCg706sg2ICAn15wQT6citbEthdQKDvTqyDYgLPI+JZsTGfNdxvbx47e1b/+iVA4JaAQLccCLwo4Pavy1eE274ut7Ilgd0FBPruxDooJiDQlxdMoC+3siWB3QUE+u7EOigmINCXF+yjiMgn1HkRINCBgEDvoAiG0JXAxxHxeVcj6ncwbvvab22MbEIBgT5h0U35lQLu5758gbwdET8u39yWBAjsKSDQ99TVdkUBgb68aj4/llvZksDuAt6QuxProJiAQF9eMJ8fy61sSWB3AW/I3Yl1UExAoC8rmJvKLHOyFYHDBAT6YdQ6KiIg0JcVSqAvc7IVgcMEBPph1DoqIiDQlxXKNejLnGxF4DABgX4YtY6KCOTTw34rMtYzhynQz9TXN4F7BAS6ZUHgrsB/oTwq4Br0R4lsQOBYAYF+rLfeaggI9MfrJNAfN7IFgUMFBPqh3DorIiDQHy+Uz47HjWxB4FABb8pDuXVWRECgP14onx2PG9mCwKEC3pSHcuusiIBAf3WhXLJWZCEb5lwCAn2uepvtMgGBLtCXrRRbEehIQKB3VAxD6UZAoL+6FC5Z62apGgiBvwQEutVA4K6AQBfo3hcEygkI9HIlM+ADBH6JiL8f0E/VLlyyVrVyxj20gEAfurwm1yjw74h4p3HfGXYT6DNU2RzLCQj0ciUz4AMEBPqrkX1uHLAIdUFgrYA35lox288gINAF+gzr3BwHExDogxXUdDYReB4RzzZpabxGfoqIt8ablhkRqC8g0OvX0Ay2F/gsIj7dvtkhWnRTmSHKaBIjCgj0EatqTtcKCPSHBb+NiPevBbY/AQLbCwj07U21WF/g3Yj4rv40dpmBm8rswqpRAtcLCPTrDbUwnoBAf7imAn289W5GgwgI9EEKaRqbCgj0hzldg77pUtMYge0EBPp2lloaR0CgC/RxVrOZTCMg0KcptYmuEHg9In5esf1Mm74WEf+ZacLmSqCKgECvUinjPFrAA1ruF/eZcfRK1B+BhQLenAuhbDadgEC/W/JfIyKPXngRINChgEDvsCiG1IWAQL9bBjeV6WJpGgQBh8+sAQJrBAT6Xa0vIuLjNYi2JUDgOAHf0I+z1lMtAYF+t16uQa+1ho12MgGBPlnBTXexgEC/S+Ua9MXLx4YEjhcQ6Meb67GGgEC/W6e3I+LHGuUzSgLzCQj0+WpuxssEBPpdJ58Xy9aOrQicIuANegq7TgsICPQXi+SStQKL1hDnFhDoc9ff7B8W+CUi/g7ofwIuWbMYCHQuINA7L5DhnSbw74h457Te++vYJWv91cSICLwgINAtCAL3Cwj0F11csuadQqBzAYHeeYEM7zQBgf4ivUvWTluKOiawTECgL3Oy1XwCAv3Fmrtkbb73gBkXExDoxQpmuIcJCPQXqX1WHLb0dESgTcCbtM3NXuMLPI+IZ+NPc/EMfVYsprIhgXMEvEnPcddr/wKfRcSn/Q/zkBG6Bv0QZp0QuE5AoF/nZ+9xBQT6X7V1Dfq469zMBhIQ6AMV01Q2FRDoAn3TBaUxAnsLCPS9hbVfVeD9iPi66uA3Hrdr0DcG1RyBPQQE+h6q2hxB4N2I+G6EiWwwB4G+AaImCOwtIND3FtZ+VQGB/lflBHrVVWzcUwkI9KnKbbIrBAT6X1juErdi4diUwFkCAv0sef32LvAkIn7rfZAHjU+gHwStGwLXCAj0a/TsO7qAZ6L/WWGBPvpKN78hBAT6EGU0iZ0EBPqfsK9FxH92MtYsAQIbCQj0jSA1M6SAQP+zrD4nhlzeJjWagDfqaBU1ny0FfoyIN7dssGhbPieKFs6w5xLwRp2r3ma7TsAT1yJ+j4g8QdCLAIHOBQR65wUyvFMFBHqE+7ifugR1TmC5gEBfbmXL+QQEukCfb9WbcVkBgV62dAZ+gIBAF+gHLDNdENhGQKBv46iVMQU8cS3ii4j4eMzymhWBsQQE+lj1NJttBQR6hPu4b7umtEZgNwGBvhuthgcQEOgCfYBlbAqzCAj0WSptni0CAj3ik4h43oJnHwIEjhUQ6Md6662WgEB3H/daK9ZopxYQ6FOX3+QfEXgaEV9OruTBLJMvANOvIyDQ69TKSI8X8Ex039CPX3V6JNAoINAb4ew2hYBAj3gjIn6ZotomSaC4gEAvXkDD31VAoHvS2q4LTOMEthQQ6Ftqams0AYEu0Edb0+YzsIBAH7i4pna1gEAX6FcvIg0QOEpAoB8lrZ+KArMH+q8R8XrFwhkzgRkFBPqMVTfnpQKzB7pHpy5dKbYj0IGAQO+gCIbQrcCTiPit29HtPzCBvr+xHghsJiDQN6PU0KAC/x10XkumJdCXKNmGQCcCAr2TQhhGtwIzB7pHp3a7LA2MwF0BgW5VEHi1wMyB7tGp3h0ECgkI9ELFMtRTBAT6Kew6JUBgrYBAXytm+9kEBPpsFTdfAkUFBHrRwhn2YQIzB/oHEfHNYdI6IkDgKgGBfhWfnScQmDnQPTp1ggVuiuMICPRxamkm+wgI9H1ctUqAwMYCAn1jUM0NJyDQhyupCREYU0Cgj1lXs9pOYOZA9/mw3TrSEoHdBbxhdyfWQXEBgV68gIZPYBYBgT5Lpc2zVUCgt8rZjwCBQwUE+qHcOisoMGuge3RqwcVqyHMLCPS562/2jwvMGugezPL42rAFga4EBHpX5TCYDgUEeodFMSQCBO4KCHSrgsCrBWYN9K8i4qnFQYBAHQGBXqdWRnqOwKyB7klr56w3vRJoFhDozXR2nERAoE9SaNMkUF1AoFevoPHvLTBroH8SEc/3xtU+AQLbCQj07Sy1NKbArIHuwSxjrmezGlhAoA9cXFPbRECgb8KoEQIE9hYQ6HsLa7+6gECvXkHjJzCJgECfpNCm2Swg0Jvp7EiAwJECAv1IbX1VFJg10N+IiF8qFsyYCcwqINBnrbx5LxF4EhG/LdlwwG18NgxYVFMaW8Cbduz6mt11Au9GxHfXNVF2b58NZUtn4LMKeNPOWnnzXiIg0Jco2YYAgS4EBHoXZTCITgUEeqeFMSwCBO4KCHSrgsDDAgLd6iBAoIyAQC9TKgM9QUCgn4CuSwIE2gQEepubveYQEOhz1NksCQwhINCHKKNJ7CTwWUR8ulPbvTfrs6H3ChkfgZcEvGktCQIPCwh0q4MAgTICAr1MqQz0BIGZA/3tiPjxBHNdEiDQKCDQG+HsNoXAzIHu8alTLHGTHElAoI9UTXPZWuDfEfHO1o0WaU+gFymUYRK4CAh0a4HAwwIC3eogQKCMgEAvUyoDPUFAoJ+ArksCBNoEBHqbm73mEPghIt6aY6p3ZumQ+6SFN+26AgK9bu2MfH+BWZ+FnrKfRMTz/Yn1QIDAVgICfStJ7YwoMHOg/zMi8ix/LwIEiggI9CKFMsxTBAT6Kew6JUCgRUCgt6jZZwaB1yPi5xkm+sAcfUOfuPimXlNAoNesm1HvLzDzg1lSV6Dvv8b0QGBTAYG+KafGBhIQ6H5DH2g5m8oMAgJ9hiqbY4uAQBfoLevGPgROExDop9HruHOBpxHxZedj3HN430dE/qPGiwCBIgICvUihDPNwgZkfzJLYAv3wJadDAtcJCPTr/Ow9roBA9w193NVtZkMKCPQhy2pSGwgIdIG+wTLSBIHjBAT6cdZ6qiXwTUS8V2vIm472q4jI8wi8CBAoIiDQixTKMA8XmPlJa4ntOvTDl5wOCVwnINCv87P3uAIC3WVr465uMxtSQKAPWVaT2kBAoAv0DZaRJggcJyDQj7PWUy2BmR/M4pB7rbVqtAT+EBDoFgKB+wUEum/o3hsESgkI9FLlMtgDBWYP9H9ERP7s4EWAQBEBgV6kUIZ5qMDsj05NbIF+6JLTGYHrBQT69YZaGE9g9gezCPTx1rQZTSAg0CcosimuFhDovqGvXjR2IHC2gEA/uwL671Hg44j4vMeBHTgmh9wPxNYVgS0EBPoWitoYTWD2+7hnPX02jLaqzWd4AW/a4Utsgg0CAl2gNywbuxA4V0Cgn+uv9z4FZr9LnG/ofa5LoyLwSgGBboEQuCsg0H1D974gUE5AoJcrmQEfIPBzROS16DO/fDbMXH1zLyngTVuybAa9s8Dsd4lzyH3nBaZ5AnsICPQ9VLVZWeBJRPxWeQIbjd1nw0aQmiFwlIA37VHS+qki4KYyf1bKZ0OVFWucBG4EvGktBQIvCrwfEV9DEejWAIFqAgK9WsWMd28B16D7hr73GtM+gV0EBPourBotLPCviPiw8Pi3GrrPhq0ktUPgIAFv2oOgdVNGwDXovqGXWawGSuC2gEC3Hgi8KOCSNYHuPUGgpIBAL1k2g95J4K2I+GGntqs167OhWsWMd3oBb9rplwCAWwLOcP8Lw2eDtwaBYgLetMUKZri7CjyPiGe79lCncZ8NdWplpAT+EPCmtRAI/CWQh9vzsLuXzwZrgEA5AYFermQGvKOAE+Icct9xeWmawL4CAn1fX63XEXDL1xdr5bOhzto1UgIOuVsDBG4JuEOcQPeGIFBawL/CS5fP4DcUcEMZgb7hctIUgeMFBPrx5nrsU8Dv5wK9z5VpVAQWCgj0hVA2G1rA7+d3y+uzYeglb3IjCnjTjlhVc1or4Pdzgb52zdieQHcCAr27khjQCQJ+PxfoJyw7XRLYVkCgb+uptZoCfj8X6DVXrlETuCUg0C2H2QX8fn7/CvDZMPs7w/zLCXjTliuZAW8s8HFEfL5xmyM057NhhCqaw1QC3rRTldtk7xH4JiLeI3NHwGeDRUGgmIA3bbGCGe7mAj9HxOubt1q/QZ8N9WtoBpMJeNNOVnDTfdpiXEYAAAhXSURBVEHgSUT8xuReAZ8NFgaBYgLetMUKZribCjgh7n7OnzxGdtN1pjEChwgI9EOYddKpgBPi7i/M9xGR/9jxIkCgkIBAL1QsQ91c4F8R8eHmrdZvUKDXr6EZTCgg0Ccsuin/T8Ad4u5fDF9FxFPrhACBWgICvVa9jHZbAXeIu9/znxGR97f3IkCgkIBAL1QsQ91UwBnuD3MK9E2XmsYIHCMg0I9x1kt/As5wf7gm/4iI/DnCiwCBQgICvVCxDHVTAWe4C/RNF5TGCJwtINDProD+zxLwDPSH5X0unLUq9UvgCgFv3Cvw7FpawBnuAr30AjZ4Ai8LCHRrYlaBH9wN7d7SuwZ91neEeZcXEOjlS2gCjQIuWbsfzjXojQvKbgTOFhDoZ1dA/2cIuGTtYXWXrJ2xIvVJYAMBgb4BoibKCbhk7eGSfRAR+Yx4LwIEigkI9GIFM9xNBAT6w4xvR8SPmyhrhACBQwUE+qHcOutEwCVrDxfCZ0Ini9QwCKwV8OZdK2b7EQQE+v1V9Bz0EVa3OUwrINCnLf3UE/fY1PvL7wz3qd8WJl9dQKBXr6Dxtwi4qcz9ap9ExPMWUPsQIHC+gEA/vwZGcLyAQL/f3ENZjl+LeiSwmYBA34xSQ4UEBPr9xfJ5UGgRGyqBlwW8ga2JGQV+jojXZ5z4K+bslq8WBIHiAgK9eAENv0nAbV/vsrlDXNNSshOBfgQEej+1MJLjBPLGKW8e112Jnvx+XqJMBkngYQGBbnXMKOA39LtV91kw4zvBnIcS8CYeqpwms1BAoL8I9W1EvL/QzmYECHQqINA7LYxh7SqQ11o/27WHWo1/FBF5sx0vAgQKCwj0wsUz9GaBjyPi8+a9x9vxtYj4z3jTMiMCcwkI9LnqbbZ/CrwVET/A+EPA4XYLgcAgAgJ9kEKaxmqB/Eb6f6v3Gm8Hh9vHq6kZTSog0CctvGn/8Zvxh5M7/H5zgx2H2ydfCKY/hoBAH6OOZrFewGH3CE9XW79u7EGgWwGB3m1pDOwAgdm/pb8dEXmTHS8CBAYQEOgDFNEUmgXyfu4ZaDP+lv6r+9k3rxs7EuhSQKB3WRaDOlAgb6jy9YH99dKVZ5/3UgnjILCRgEDfCFIzpQWeRsSXpWewfvCuPV9vZg8CXQsI9K7LY3AHCswU6q49P3Bh6YrAUQIC/Shp/VQQyDPf8z7vo/+m/kFEfFOhIMZIgMByAYG+3MqWcwg8ubmv+XuDTjevPc85ehEgMJiAQB+soKazmUCeLJcPcfn7Zi320dAXEZH3svciQGAwAYE+WEFNZ3OBDL/PBjoM79rzzZeIBgn0ISDQ+6iDUfQtkIeoM9jzT6Xf1/Na87zOPv/kuQH5t9u89r3WjI5As4BAb6az44QCPQf7TzeB/YvwnnBlmjKBiBDolgGBNoG8zC2/sb/ZtvtVe13C+/a376satDMBAvUFBHr9GprBuQJ5qVsGe55Et8fh+O9vHTa/BPi5M9Y7AQJdCgj0LstiUEUF3r0J9gz5d1bOIS8nu/zWnYfNhfdKQJsTmF1AoM++Asx/T4EM+HwATP7J/85XBv3t8L4Ed4a4FwECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoFBHoznR0JECBAgEA/AgK9n1oYCQECBAgQaBYQ6M10diRAgAABAv0ICPR+amEkBAgQIECgWUCgN9PZkQABAgQI9CMg0PuphZEQIECAAIFmAYHeTGdHAgQIECDQj4BA76cWRkKAAAECBJoF/h9LvzgiHsa2MgAAAABJRU5ErkJggg=='

img64 = img64.replace('data:image/png;base64,', '')

imgbytes = base64.b64decode(img64)

In [ ]:
img = Image.open(io.BytesIO(imgbytes))

In [ ]:
x_coordinates = []
y_coordinates = []

In [ ]:
img.size

In [ ]:
img.thumbnail((imsize, imsize))

In [ ]:
img_array = np.array(img)
bin_img_array = [[1 if pixel[3] == 255 else 0 for pixel in row] for row in img_array]

In [ ]:
contours = find_contours(bin_img_array, 0)

In [ ]:
for contour_ in contours:
    for (x, y) in contour_:
        # Invert x and y to be compatible with back-end pipeline
        y_coordinates.append(imsize - x)
        x_coordinates.append(y)

In [ ]:
def binary_image(im_size, x_coordinates, y_coordinates):
    x_coordinates = np.array(x_coordinates, dtype=float)
    y_coordinates = np.array(y_coordinates, dtype=float)

    x_coordinates -= min(x_coordinates)
    y_coordinates -= min(y_coordinates)

    x_max = max(x_coordinates)
    y_max = max(y_coordinates)

    if x_max >= y_max:
        ratio = im_size / x_max

    else:
        ratio = im_size / y_max
        
    print(ratio)
    
    x_coordinates *= ratio
    y_coordinates *= ratio

    img = np.zeros((im_size, im_size), dtype=np.uint8)

    x, y = polygon(x_coordinates, y_coordinates)
    img[x, y] = 1
    
    return img


def binary_image_center(im_size, x_coordinates, y_coordinates, y_centroid):
    # In that space, y and x axis are inverted
    x_centroid = y_centroid
    
    x_coordinates = np.array(x_coordinates)
    y_coordinates = np.array(y_coordinates)

    x_coordinates -= min(x_coordinates)
    y_coordinates -= min(y_coordinates)

    x_max = max(x_coordinates)
    y_max = max(y_coordinates)

    ratio = im_size / y_max

    x_coordinates *= ratio
    y_coordinates *= ratio

    x_coordinates = np.array([min(im_size - 1, max(0, x + im_size / 2 - x_centroid)) for x in x_coordinates])
    y_coordinates = np.array([min(im_size - 1, max(0, y)) for y in y_coordinates])
    
    img = np.zeros((im_size, im_size), dtype=np.uint8)

    x, y = polygon(x_coordinates, y_coordinates)
    
    img[x, y] = 1
    
    return img


def unique_orientation_coords(bin_image, x_coords, y_coords):
    props = regionprops(bin_image)

    angle = -props[0].orientation
    cos = np.cos(angle)
    sin = np.sin(angle)

    x_coords_rot = []
    y_coords_rot = []

    for i in range(len(x_coords)):
        x = x_coords[i]
        y = y_coords[i]

        x_coords_rot.append(x * cos - y * sin)
        y_coords_rot.append(y * cos + x * sin)

        
    return x_coords_rot, y_coords_rot, angle

def centroid_y(bin_img, im_size):
    y_counts = np.sum(bin_img, axis=1)
    indices = np.arange(im_size)

    return sum(y_counts * indices) / sum(y_counts)

In [ ]:
plt.imshow(binary_image(imsize, x_coordinates, y_coordinates))

In [ ]:
coordinates = (x_coordinates, y_coordinates)

bin_image_orig = binary_image(imsize, *coordinates)

x_coords_rot, y_coords_rot, angle = unique_orientation_coords(bin_image_orig, coordinates[0], coordinates[1])
bin_image_rot = binary_image(imsize, x_coords_rot, y_coords_rot)

centroid = centroid_y(bin_image_rot, imsize)
bin_image_center = binary_image_center(imsize, x_coords_rot, y_coords_rot, centroid)

skeleton, distance = medial_axis(bin_image_center, return_distance=True)

plt.imshow(distance)